In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ["LLAMA_INDEX_CACHE_DIR"] = '/mnt/nfs/renjiyuan/HF_cache'
os.environ['HF_HOME'] = '/mnt/nfs/renjiyuan/HF_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/mnt/nfs/renjiyuan/HF_cache'

In [2]:
# Setup AzureOpenAI Agent
from llama_index.llms.azure_openai import AzureOpenAI

llm = AzureOpenAI(
    engine="gpt-4",
    model="gpt-4",
    temperature=0.0
)

/home/renjiyuan/miniconda3/envs/rag_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# test llm api
completion_response = llm.complete("To infinity, and")
print('Test api:\n', completion_response)

Test api:
 beyond! 

This phrase is the iconic catchphrase of Buzz Lightyear, a character from the "Toy Story" franchise produced by Pixar Animation Studios and released by Walt Disney Pictures. Buzz Lightyear is a space ranger action figure and one of the main characters in the series, known for his bravery and somewhat delusional belief that he is a real space ranger rather than a toy. The phrase encapsulates his adventurous spirit and has become synonymous with taking on challenges without limits.


In [4]:
from llama_index.core.tools.download import download_tool
from llama_index.core.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
from baidu_search_tool import BaiduSearchToolSpec

# setup GoogleSearchToolSpec
baidu_spec = BaiduSearchToolSpec()

# prepare tools
tools = LoadAndSearchToolSpec.from_defaults(
    baidu_spec.to_tool_list()[0],
).to_tool_list()

In [5]:
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core import Settings

# setting the embed model, for default embed model is openai.
# If you have the key of openai, you can set it in the env and skip this step.
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
Settings.llm = llm

In [6]:
from llama_index.core.agent import AgentRunner, ReActAgent
# this method will automatically pick the best agent depending on the LLM
# agent = AgentRunner.from_llm(tools, llm=llm, verbose=True)

# or
# from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

In [7]:
# test 
agent.chat("中文里面山高水长是什么意思？")

Thought: The current language of the user is Chinese. I need to use a tool to help me answer the question about the meaning of the phrase "山高水长" in Chinese.
Action: baidu_search
Action Input: {'query': '山高水长 意思'}
[ERROR] https://www.baidu.com/baidu?wd=%E5%B1%B1%E9%AB%98%E6%B0%B4%E9%95%BF%20%E6%84%8F%E6%80%9D&tn=monline_dg&ie=utf-8 请求失败: HTTPSConnectionPool(host='www.baidu.com', port=443): Read timed out. (read timeout=60)
Observation: Content loaded! You can now search the information using read_baidu_search
Thought: Now that the content is loaded, I can use the read_baidu_search tool to find the meaning of the phrase "山高水长".
Action: read_baidu_search
Action Input: {'query': '山高水长是什么意思'}
Observation: "山高水长"是一个汉语成语，拼音为shān gāo shuǐ cháng，意思是像山一样高耸，如水一般长流。这个成语原本用来比喻人的风范或声誉像高山一样永远存在，后来也用来比喻恩德深厚。
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: "山高水长"是一个汉语成语，表示像山一样高耸，如水一样长流。它原本用来形容人的风范或声誉像高山一样永存，后来也用来形容恩德深厚。


AgentChatResponse(response='"山高水长"是一个汉语成语，表示像山一样高耸，如水一样长流。它原本用来形容人的风范或声誉像高山一样永存，后来也用来形容恩德深厚。', sources=[ToolOutput(content='Content loaded! You can now search the information using read_baidu_search', tool_name='baidu_search', raw_input={'args': (), 'kwargs': {'query': '山高水长 意思'}}, raw_output='Content loaded! You can now search the information using read_baidu_search', is_error=False), ToolOutput(content='"山高水长"是一个汉语成语，拼音为shān gāo shuǐ cháng，意思是像山一样高耸，如水一般长流。这个成语原本用来比喻人的风范或声誉像高山一样永远存在，后来也用来比喻恩德深厚。', tool_name='read_baidu_search', raw_input={'args': (), 'kwargs': {'query': '山高水长是什么意思'}}, raw_output='"山高水长"是一个汉语成语，拼音为shān gāo shuǐ cháng，意思是像山一样高耸，如水一般长流。这个成语原本用来比喻人的风范或声誉像高山一样永远存在，后来也用来比喻恩德深厚。', is_error=False)], source_nodes=[], is_dummy_stream=False)